In [1]:
# PYTHON IMPORTS
import os
import copy
from tqdm.notebook import trange, tqdm

# IMAGE IMPORTS 
from PIL import Image
import cv2

# DATA IMPORTS 
import random
import h5py
import numpy as np
import glob

# PLOTTING
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# NEURAL NETWORK
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.transforms import ToPILImage, GaussianBlur
from torchvision.transforms import Compose, RandomCrop, ToTensor, Normalize
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.models as models

# MY OWN CLASSES
from SquareLocator import *
Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
data_dir = r"C:\Users\franc\OneDrive - University Of Houston\AAA_RECTDNN\data/"

templates_dir = f"{data_dir}templates/"
tempfiles_dir = "tempfiles/"

boundary_shapefile = f"{templates_dir}HCAD_Harris_County_Boundary.shp"
boundary_points    = f'{tempfiles_dir}boundary_points.shp'
roads_points       = f'{tempfiles_dir}roads_points.shp'

roads_fn = f"data/Roads/TexasHighways.shp"
model_checkpoint = f"{data_dir}SquareLocator/checkpoint_072823.pth"

input_folder = f"../data/HistoricalFIRMS/"
output_dir   = f"../data/ZZZ_Squares"

tensor = transforms.Compose([
        transforms.ToTensor(),
    ])

In [3]:
# Initialize model
model = SquareLocator(finalpadding=1)
model.load_state_dict(torch.load(model_checkpoint)['model_state_dict'])
model = model.to("cuda")

In [6]:
images = glob.glob(input_folder+"*.tif")
images.append(glob.glob(input_folder+"*.png"))
images.append(glob.glob(input_folder+"*.jpg"))

In [7]:
# Iterate over the files in the input folder
for image in tqdm(images):
    # Open the image file
    filename = os.path.basename(image)
    image_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_dir, filename)

    # INPUT IMAGE AND PREP
    image = np.asarray(cv2.imread(image_path, 0))
    shape = image.shape
    image = cv2.resize(image, (512, 512))    
    image_prep = tensor(image).unsqueeze(0).to("cuda")

    # PROCESS IMAGE
    outputs = model(image_prep)
    outputs = outputs[0, 1, :, :].detach().cpu().numpy()

    # RESIZE AND OUTPUT 
    outputs_resized = cv2.resize(outputs, shape)
    cv2.imwrite(output_path, outputs_resized * 255)

  0%|          | 0/493 [00:00<?, ?it/s]

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src data type = 17 is not supported
>  - Expected Ptr<cv::UMat> for argument 'src'
